## Missed Trash Pickups

In this data question you will be working data of service request related to missed trash pickups from hubNashville, Metro Nashville government's comprehensive customer service system (https://hub.nashville.gov).

As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. One category of liquidated damages is related to chronic problems in any category of service at the same premises. A chronic problem is defined as more than one missed pickup for any address. The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

After determining the total amount of damages, you can look at other questions:

* What other types of complaints are there?
* Make a heat map that shows the most total missed pick ups and another that shows the total fines, each by zip code.
* How do metro crews compare to the contractor's performance?
* How much does each trash hauler owe?
* What were to total missed pickup by route?

Before you begin, explore the data.  Cleaning and preparing the data for analysis is an important and necessary step.  
Planning and communication are vital to success. This data and analysis are based on a real-world project.

In [114]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import re


In [115]:
trash = pd.read_csv('./trash_hauler_report.csv')
trash

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,11/01/17,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/01/19,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1.781137e+06,632448.551144
20222,267126,11/01/19,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1.749711e+06,669201.601569
20223,267130,11/01/19,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1.770293e+06,674936.303809
20224,267134,11/01/19,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1.785225e+06,627146.400187


In [116]:
trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 1.7+ MB


In [117]:
trash['Date Opened'] = pd.to_datetime(trash['Date Opened'], format = 'mixed')

In [120]:
trash['Zip Code'] = trash['Zip Code'].astype('Int64')

In [121]:
trash

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,2017-11-01,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20.0,1.710186e+06,664205.101066
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,2019-11-01,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013,RED RIVER,4502,32.0,1.781137e+06,632448.551144
20222,267126,2019-11-01,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206,METRO,9508,6.0,1.749711e+06,669201.601569
20223,267130,2019-11-01,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214,RED RIVER,1502,15.0,1.770293e+06,674936.303809
20224,267134,2019-11-01,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013,RED RIVER,4502,32.0,1.785225e+06,627146.400187


In [122]:
trash['Council District'].unique()

array([ 2.,  1., 20., 16., 34., 21., 33., 19.,  3.,  5., 15., 25., 23.,
       26., 18., 24., 22., 29., 31., 28., 13., 17., 30., 27.,  4.,  6.,
       32., 14.,  7.,  8., nan, 11., 12.,  9., 35., 10.])

In [123]:
trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Request Number    20226 non-null  int64         
 1   Date Opened       20226 non-null  datetime64[ns]
 2   Request           20226 non-null  object        
 3   Description       20195 non-null  object        
 4   Incident Address  20217 non-null  object        
 5   Zip Code          20151 non-null  Int64         
 6   Trash Hauler      19325 non-null  object        
 7   Trash Route       19279 non-null  object        
 8   Council District  20177 non-null  float64       
 9   State Plan X      20198 non-null  float64       
 10  State Plan Y      20198 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 1.7+ MB


In [124]:
trash.value_counts('Request ')

Request 
Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2629
Trash Collection Complaint               2312
Damage to Property                        257
Name: count, dtype: int64

In [125]:
trash = trash.rename(columns = {'Request ':'Request'})

In [126]:
trash['Request'].unique()

array(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup',
       'Trash Collection Complaint', 'Damage to Property'], dtype=object)

In [127]:
potential_miss = r'.[Mm][Ii][Ss]+.'

In [128]:
missed = trash.loc[trash['Description'].str.contains(potential_miss, na=False) | trash['Request'].str.contains(potential_miss, na=False)]

In [129]:
missed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20.0,1.710186e+06,664205.101066
7,25327,2017-11-01,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203,METRO,9208,21.0,1.731459e+06,666013.601229
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,2019-11-01,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013,RED RIVER,4502,32.0,1.781137e+06,632448.551144
20222,267126,2019-11-01,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206,METRO,9508,6.0,1.749711e+06,669201.601569
20223,267130,2019-11-01,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214,RED RIVER,1502,15.0,1.770293e+06,674936.303809
20224,267134,2019-11-01,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013,RED RIVER,4502,32.0,1.785225e+06,627146.400187


In [130]:
missed['Council District'] = missed['Council District'].astype('Int64')

C:\Users\user\AppData\Local\Temp\ipykernel_15180\2357402208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missed['Council District'] = missed['Council District'].astype('Int64')


In [131]:
missed.loc[missed['Incident Address'].isna()]

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
625,33128,2017-12-13,Trash - Curbside/Alley Missed Pickup,Missed.,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
875,35857,2017-12-29,Trash - Curbside/Alley Missed Pickup,daughters car parked in front,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
1154,39689,2018-01-17,Trash - Curbside/Alley Missed Pickup,Trash pick up not done for Tuesday 1/16/18,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
1346,41604,2018-01-24,Trash - Curbside/Alley Missed Pickup,cart still out,NaN,37218,RED RIVER,3203,1,1.715186e+06,682289.961678
2794,58920,2018-04-06,Trash - Curbside/Alley Missed Pickup,missed- trash,NaN,37206,METRO,9503,5,1.747402e+06,674741.056063
2839,59517,2018-04-10,Trash - Curbside/Alley Missed Pickup,Trash was not pick up last week.,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
3121,62051,2018-04-22,Trash - Curbside/Alley Missed Pickup,They forgot to pick up trash from apartment co...,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
11049,157096,2019-03-20,Trash - Curbside/Alley Missed Pickup,Trash has not been picked up on the whole street.,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN


In [132]:
missed.loc[1346, 'Incident Address'] = '4032 Stewarts Ln'

In [133]:
missed.loc[2794, 'Incident Address'] = '1040 Chicamauga Ave'

In [134]:
missed.loc[missed['Incident Address'].isna()]

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
625,33128,2017-12-13,Trash - Curbside/Alley Missed Pickup,Missed.,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
875,35857,2017-12-29,Trash - Curbside/Alley Missed Pickup,daughters car parked in front,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
1154,39689,2018-01-17,Trash - Curbside/Alley Missed Pickup,Trash pick up not done for Tuesday 1/16/18,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
2839,59517,2018-04-10,Trash - Curbside/Alley Missed Pickup,Trash was not pick up last week.,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
3121,62051,2018-04-22,Trash - Curbside/Alley Missed Pickup,They forgot to pick up trash from apartment co...,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
11049,157096,2019-03-20,Trash - Curbside/Alley Missed Pickup,Trash has not been picked up on the whole street.,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN


In [135]:
missed = missed.dropna(subset=['Incident Address'])

In [136]:
missed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1,1.721259e+06,685444.799565
2,25276,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20,1.707027e+06,659887.471571
3,25307,2017-11-01,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2,1.735692e+06,685027.245923
4,25312,2017-11-01,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20,1.710186e+06,664205.101066
7,25327,2017-11-01,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203,METRO,9208,21,1.731459e+06,666013.601229
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,2019-11-01,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013,RED RIVER,4502,32,1.781137e+06,632448.551144
20222,267126,2019-11-01,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206,METRO,9508,6,1.749711e+06,669201.601569
20223,267130,2019-11-01,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214,RED RIVER,1502,15,1.770293e+06,674936.303809
20224,267134,2019-11-01,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013,RED RIVER,4502,32,1.785225e+06,627146.400187


In [137]:
missed['Clean Address'] = missed['Incident Address'].str.split(',').str[0]

C:\Users\user\AppData\Local\Temp\ipykernel_15180\404135838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missed['Clean Address'] = missed['Incident Address'].str.split(',').str[0]


In [138]:
missed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,Clean Address
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1,1.721259e+06,685444.799565,4028 Clarksville Pike
2,25276,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20,1.707027e+06,659887.471571,6528 Thunderbird Dr
3,25307,2017-11-01,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2,1.735692e+06,685027.245923,2603 old matthews rd
4,25312,2017-11-01,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20,1.710186e+06,664205.101066,604 croley dr
7,25327,2017-11-01,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203,METRO,9208,21,1.731459e+06,666013.601229,1816 Jo Johnston Ave
...,...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,2019-11-01,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013,RED RIVER,4502,32,1.781137e+06,632448.551144,2731 Murfreesboro Pike
20222,267126,2019-11-01,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206,METRO,9508,6,1.749711e+06,669201.601569,1621 Long Ave
20223,267130,2019-11-01,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214,RED RIVER,1502,15,1.770293e+06,674936.303809,2943 Windemere Cir
20224,267134,2019-11-01,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013,RED RIVER,4502,32,1.785225e+06,627146.400187,3325 Murfreesboro Pike


In [139]:
missed['Clean Address'] = missed['Clean Address'].str.title()

C:\Users\user\AppData\Local\Temp\ipykernel_15180\456157262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missed['Clean Address'] = missed['Clean Address'].str.title()


In [140]:
missed

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,Clean Address
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1,1.721259e+06,685444.799565,4028 Clarksville Pike
2,25276,2017-11-01,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20,1.707027e+06,659887.471571,6528 Thunderbird Dr
3,25307,2017-11-01,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2,1.735692e+06,685027.245923,2603 Old Matthews Rd
4,25312,2017-11-01,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20,1.710186e+06,664205.101066,604 Croley Dr
7,25327,2017-11-01,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203,METRO,9208,21,1.731459e+06,666013.601229,1816 Jo Johnston Ave
...,...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,2019-11-01,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013,RED RIVER,4502,32,1.781137e+06,632448.551144,2731 Murfreesboro Pike
20222,267126,2019-11-01,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206,METRO,9508,6,1.749711e+06,669201.601569,1621 Long Ave
20223,267130,2019-11-01,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214,RED RIVER,1502,15,1.770293e+06,674936.303809,2943 Windemere Cir
20224,267134,2019-11-01,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013,RED RIVER,4502,32,1.785225e+06,627146.400187,3325 Murfreesboro Pike


In [141]:
missed_fines = missed['Clean Address'].value_counts().to_frame()

In [142]:
missed_fines = missed_fines.reset_index()

In [143]:
missed_fines

,Clean Address,count
0,110 George L Davis Blvd,25
1,12546 Old Hickory Blvd,21
2,5135 Hickory Hollow Pkwy,20
3,6007 Obrien Ave,20
4,802 Crescent Rd,19
...,...,...
10912,115 Colemont Ct,1
10913,6726 Pennywell Dr,1
10914,2709 Brightwood Ave,1
10915,1904 Meharry Blvd,1


In [144]:
missed_fines = missed_fines[missed_fines['count'] >= 2]

In [145]:
missed_fines

,Clean Address,count
0,110 George L Davis Blvd,25
1,12546 Old Hickory Blvd,21
2,5135 Hickory Hollow Pkwy,20
3,6007 Obrien Ave,20
4,802 Crescent Rd,19
...,...,...
3065,2626 Bluefield Ave,2
3066,3572 Shakertown Rd,2
3067,1776 Hillmont Dr,2
3068,117 Lafayette Ct,2


In [146]:
missed_fines['Fines'] = (missed_fines['count'] - 1) * 200

C:\Users\user\AppData\Local\Temp\ipykernel_15180\3489555802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missed_fines['Fines'] = (missed_fines['count'] - 1) * 200


In [147]:
missed_fines

,Clean Address,count,Fines
0,110 George L Davis Blvd,25,4800
1,12546 Old Hickory Blvd,21,4000
2,5135 Hickory Hollow Pkwy,20,3800
3,6007 Obrien Ave,20,3800
4,802 Crescent Rd,19,3600
...,...,...,...
3065,2626 Bluefield Ave,2,200
3066,3572 Shakertown Rd,2,200
3067,1776 Hillmont Dr,2,200
3068,117 Lafayette Ct,2,200


In [148]:
total_fines = missed_fines['Fines'].sum()

In [149]:
total_fines

np.int64(1175200)